# Apple - Corporate Social Responsibility Community Program Impact

```SQL
DROP TABLE IF EXISTS  dim_community;
DROP TAbLE IF EXISTS fct_philanthropic_initiatives;

CREATE TABLE dim_community (
    community_id INT,
    community_name VARCHAR,
    region VARCHAR
);

CREATE TABLE fct_philanthropic_initiatives (
    program_id INT,
    community_id INT,
    event_date DATE,
    participants INT,
    program_name VARCHAR
);

INSERT INTO dim_community (community_id, community_name, region)
VALUES
    (101, 'San Francisco', 'West'),
    (102, 'New York', 'East'),
    (103, 'Chicago', 'Midwest'),
    (104, 'Austin', 'South'),
    (105, 'Boston', 'East'),
    (106, 'Los Angeles', 'West'),
    (107, 'Seattle', 'West'),
    (108, 'Miami', 'South'),
    (109, 'Denver', 'West'),
    (110, 'Philadelphia', 'East');

INSERT INTO fct_philanthropic_initiatives (program_id, community_id, event_date, participants, program_name)
VALUES
    (1, 101, '2024-01-03', 50, 'iPad Learning Workshop'),
    (1, 102, '2024-01-15', 80, 'iPad Learning Workshop'),
    (2, 103, '2024-01-18', 65, 'Mac Coding Camp'),
    (2, 101, '2024-01-22', 40, 'Mac Coding Camp'),
    (3, 102, '2024-02-05', 90, 'Apple Community Health'),
    (3, 103, '2024-02-15', 70, 'Apple Community Health'),
    (4, 104, '2024-02-10', 55, 'Tech For Kids'),
    (4, 104, '2024-02-20', 100, 'Tech For Kids'),
    (5, 101, '2024-03-02', 120, 'Green Initiative'),
    (5, 105, '2024-03-05', 95, 'Green Initiative'),
    (6, 102, '2024-03-06', 60, 'Digital Safety Seminar'),
    (6, 103, '2024-03-07', 80, 'Digital Safety Seminar');

SELECT * FROM dim_community;
SELECT * FROM fct_philanthropic_initiatives;
```

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_community = pd.read_csv('Data/002/dim_community.csv')
df_antropic = pd.read_csv('Data/002/fct_anthropic_initiatives.csv', parse_dates=['event_date'])

df_community.head()

,community_id,community_name,region
0,101,San Francisco,West
1,102,New York,East
2,103,Chicago,Midwest
3,104,Austin,South
4,105,Boston,East


In [3]:
df_antropic.head()

,program_id,community_id,event_date,participants,program_name
0,1,101,2024-01-03,50,iPad Learning Workshop
1,1,102,2024-01-15,80,iPad Learning Workshop
2,2,103,2024-01-18,65,Mac Coding Camp
3,2,101,2024-01-22,40,Mac Coding Camp
4,3,102,2024-02-05,90,Apple Community Health


# Pregunta 1

### El equipo de Responsabilidad Social Corporativa de Apple desea un informe resumido de las iniciativas filantrópicas en enero de 2024. Por favor, elabore un informe que agregue (sume) el número de participantes por comunidad y por programa.

In [6]:
df_merge = df_community.merge(df_antropic, on='community_id', how='inner')

df_enero = df_merge[
    (df_merge['event_date'].between('2024-01-01','2024-01-31'))
]

respuesta1 = df_enero.groupby(['community_name','program_name'])['participants'].sum().reset_index(name='sum_participants')

respuesta1

,community_name,program_name,sum_participants
0,Chicago,Mac Coding Camp,65
1,New York,iPad Learning Workshop,80
2,San Francisco,Mac Coding Camp,40
3,San Francisco,iPad Learning Workshop,50


```SQL
SELECT
    c.community_name,
    p.program_name,
    SUM(p.participants) AS sum_participants
FROM dim_community c
INNER JOIN fct_philanthropic_initiatives p ON c.community_id = p.community_id
WHERE p.event_date BETWEEN '2024-01-01' AND '2024-01-31'
GROUP BY c.community_name, p.program_name
```

# Pregunta 2

### El equipo está revisando la ejecución de los programas filantrópicos de febrero de 2024. Para cada iniciativa, proporcione los detalles junto con la fecha de evento más temprana (la más antigua) registrada dentro de cada campaña del programa para comprender los tiempos de inicio.

In [7]:
df_feb = df_antropic[(df_antropic['event_date'].between('2024-02-01','2024-02-29'))].copy()

respuesta2 = df_feb.groupby('program_name')['event_date'].min().reset_index(name='earliest_event_date')

respuesta2

,program_name,earliest_event_date
0,Apple Community Health,2024-02-05
1,Tech For Kids,2024-02-10


```SQL
SELECT
    program_name,
    MIN(event_date) AS earliest_event_date
FROM fct_philanthropic_initiatives
WHERE event_date BETWEEN '2024-02-01' AND '2024-02-29'
GROUP BY program_name;
```

# Pregunta 3

### Para un análisis refinado de las iniciativas realizadas durante la primera semana de marzo de 2024, incluya para cada programa la cifra máxima de participación registrada en cualquier evento. Esta información ayudará a destacar los niveles más altos de compromiso (engagement) dentro de cada campaña.

In [10]:
df_w1 = df_antropic[(df_antropic['event_date'].between('2024-03-01','2024-03-07'))]

respuesta3 = df_w1.groupby('program_name')['participants'].max().reset_index(name='max_participants_per_program')

respuesta3

,program_name,max_participants_per_program
0,Digital Safety Seminar,80
1,Green Initiative,120


```SQL
SELECT
    program_name,
    MAX(participants) AS max_participants_per_program
FROM fct_philanthropic_initiatives
WHERE event_date BETWEEN  '2024-03-01' AND '2024-03-07'
GROUP BY program_name;
```